# Appendix: Import needed modules 

In [1]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kennd/Downloads/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [2]:
## Import all modules needed
## lem is lemmatizing the words
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(min_df=3,max_df=20,stop_words=stops,ngram_range=(1,1))
tfvectorizer = TfidfVectorizer(stop_words=stops,max_df=20, min_df=3,ngram_range=(1,1))

import pandas as pd
df = pd.read_csv('C:/Users/kennd/Desktop/yelp_collate_test.csv')

# Download data and munge munge

In [3]:
df2 = df[['sentence','category']]

In [4]:
# df = df.drop(['wait','ambience','food','service','value','none'],axis=1)

In [5]:
# df2 = df.fillna(0)

In [6]:
df2['s_lower'] = df2['sentence'].apply(lambda x: x.lower())

C:\Users\kennd\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
## lemmatizing and spell checking the words
## this  pre-processing of text is needed before feeding it into tokenizing by word
df2['s_spell_lem'] = df2['s_lower'].apply(lem)

C:\Users\kennd\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [8]:
df3 = df2[['s_spell_lem','category']]

In [9]:
corp = list(df3.s_spell_lem)

In [10]:
df3.head(10)

,s_spell_lem,category
0,three large dreaded and fried fish file but a...,value
1,needle to say he had no complaint and brought...,value
2,i brought home half the sandwich,value
3,another good meal there,value
4,most food for your dollar i highly recommend,value
5,this place is a neighborhood standard that fo...,value
6,but the fact is you get a decent meal for a d...,value
7,rocky is worth going to,value
8,you will definitely find something you like a...,value
9,decent price,value


# Feature Generation

In [11]:
## Gets the count of each word in each sentence
feature = vectorizer.fit_transform(corp)

In [12]:
## Transform into word counts into pandas data frame
featuredf = pd.DataFrame(feature.A, columns=vectorizer.get_feature_names())
# featuredf.drop(featuredf.loc[:,('075', '1245pm', '1950s', '1980s', '2013', '715', '750', 'ab','breakfastbut','burgher','barrestaurant','berg','chekmar','checked','donatucci','dinnersized','divethis','flavorheat','restaurantits','wholewheatomgit','vewy','steelersravens','le','tamoley')],axis=1,inplace=True)

In [13]:
# ## Crude normalization of features
# featuredf2 = featuredf.loc[:,featuredf.sum()>= 3]
# featuredf2 = featuredf2.loc[:,featuredf2.sum()<= 20]

In [14]:
df5 = pd.concat((df3,featuredf),axis=1)

In [15]:
X = df5.iloc[:,2:]
Y = df5.iloc[:,1]

In [16]:
df5.head()

,s_spell_lem,category,ago,almost,also,although,always,american,another,area,...,well,went,whole,wing,without,working,worth,would,year,yes
0,three large dreaded and fried fish file but a...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,needle to say he had no complaint and brought...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,i brought home half the sandwich,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,another good meal there,value,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,most food for your dollar i highly recommend,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Y.value_counts()

food        162
others       76
value        55
service      46
ambiance     39
Name: category, dtype: int64

# Building the model and in-training-set accuracy

In [18]:
from sklearn.cross_validation import train_test_split 

In [19]:
##Split into train and test at 75/25
train, test = train_test_split(df5,test_size = 0.25)

In [20]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [22]:
from sklearn.metrics import confusion_matrix

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,nb_Y_pred,labels=['ambience','service','food','value','others']))

## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcl_Y_pred,labels=['ambience','service','food','value','others']))

## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=1).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcrbf_Y_pred,labels=['ambience','service','food','value','others']))

## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,log_Y_pred,labels=['ambience','service','food','value','others']))

## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,dt_Y_pred,labels=['ambience','service','food','value','others']))

## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,rf_Y_pred,labels=['ambience','service','food','value','others']))

NB Accuracy:  0.589473684211
[[ 0  0  0  0  0]
 [ 0  7  1  0  2]
 [ 0  0 35  2  6]
 [ 0  0  4  5  3]
 [ 0  0  9  0  7]]
SVC Linear Accuracy:  0.557894736842
[[ 0  0  0  0  0]
 [ 0  7  2  0  1]
 [ 0  1 32  1  7]
 [ 0  0  3  7  2]
 [ 0  0  9  0  6]]
SVC RBF Accuracy:  0.484210526316
[[ 0  0  0  0  0]
 [ 0  2  8  0  0]
 [ 0  0 41  2  0]
 [ 0  0 10  2  1]
 [ 0  0 17  0  1]]
Logistc Accuracy:  0.568421052632
[[ 0  0  0  0  0]
 [ 0  7  1  0  2]
 [ 0  0 37  1  5]
 [ 0  0  5  6  1]
 [ 0  0 12  0  4]]
Decision Tree Accuracy:  0.536842105263
[[ 0  0  0  0  0]
 [ 0  8  1  0  1]
 [ 0  2 26  5  9]
 [ 0  0  2  9  2]
 [ 0  0  9  0  7]]
Random Forests Accuracy:  0.557894736842
[[ 0  0  0  0  0]
 [ 0  8  1  0  1]
 [ 0  2 30  4  6]
 [ 0  0  2  8  3]
 [ 0  2  8  0  5]]


In [112]:
## NB In-training-set accuracy
NB_Y_PRED = clf.predict(X_test)
np.mean(NB_Y_PRED == np.array(Y_test))

0.50526315789473686

In [113]:
from sklearn.metrics import confusion_matrix
sklearn.metrics.confusion_matrix(Y_test,NB_Y_PRED,labels=['ambience','service','food','value','others'])

array([[ 0,  0,  0,  0,  0],
       [ 0,  6,  6,  1,  0],
       [ 0,  1, 28,  4,  3],
       [ 0,  0,  9,  5,  1],
       [ 0,  2,  7,  0,  9]])

In [114]:
## SVC In-training-set accuracy
SVC_Y_PRED = svc.predict(X_test)
np.mean(SVC_Y_PRED == np.array(Y_test))

0.55789473684210522

In [115]:
import sklearn.metrics
sklearn.metrics.confusion_matrix(Y_test,SVC_Y_PRED,labels=['ambience','service','food','value','others'])

array([[ 0,  0,  0,  0,  0],
       [ 0,  5,  6,  1,  0],
       [ 0,  0, 33,  2,  2],
       [ 0,  0,  7,  6,  1],
       [ 0,  1,  8,  0,  7]])

In [116]:
import sklearn.linear_model

In [117]:
log = sklearn.linear_model.LogisticRegression().fit(X_train,Y_train)

In [118]:
LOG_Y_PRED = log.predict(X_test)
np.mean(LOG_Y_PRED == np.array(Y_test))

0.54736842105263162

In [119]:
sklearn.metrics.confusion_matrix(Y_test,LOG_Y_PRED,labels=['ambience','service','food','value','others'])

array([[ 0,  0,  0,  0,  0],
       [ 0,  5,  7,  0,  0],
       [ 0,  0, 33,  2,  1],
       [ 0,  0,  8,  6,  0],
       [ 0,  0, 12,  0,  8]])

In [120]:
import sklearn.tree

In [121]:
dt = sklearn.tree.DecisionTreeClassifier().fit(X_train,Y_train)

In [122]:
DT_Y_PRED = dt.predict(X_test)

In [123]:
'{0:.3f}'.format(np.mean(DT_Y_PRED == np.array(Y_test)))

'0.453'

In [124]:
sklearn.metrics.confusion_matrix(Y_test,LOG_Y_PRED,labels=['ambience','service','food','value','others'])

array([[ 0,  0,  0,  0,  0],
       [ 0,  5,  7,  0,  0],
       [ 0,  0, 33,  2,  1],
       [ 0,  0,  8,  6,  0],
       [ 0,  0, 12,  0,  8]])

In [125]:
import sklearn.ensemble

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=100).fit(X_train,Y_train)
RF_Y_PRED = rf.predict(X_test)

In [133]:
np.mean(RF_Y_PRED == np.array(Y_test))

0.49473684210526314

In [134]:
sklearn.metrics.confusion_matrix(Y_test,RF_Y_PRED,labels=['ambience','service','food','value','others'])

array([[ 0,  0,  0,  0,  0],
       [ 0,  5,  5,  0,  2],
       [ 0,  2, 26,  3,  5],
       [ 0,  0,  8,  6,  0],
       [ 0,  2,  7,  1,  8]])

# Sentiment Scoring

In [330]:
## Modified off-the-shelf sentiment analyzer
import nltk.sentiment.util
def demo_liu_hu_lexicon2(sentence, plot=False):
    """
    Basic example of sentiment classification using Liu and Hu opinion lexicon.
    This function simply counts the number of positive, negative and neutral words
    in the sentence and classifies it depending on which polarity is more represented.
    Words that do not appear in the lexicon are considered as neutral.

    :param sentence: a sentence whose polarity has to be classified.
    :param plot: if True, plot a visual representation of the sentence polarity.
    """
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral

    return (pos_words - neg_words)
#     if pos_words > neg_words:
#         return 'Positive'
#     elif pos_words < neg_words:
#         return 'Negative'
#     elif pos_words == neg_words:
#         return 'Neutral'

    if plot == True:
        _show_plot(x, y, x_labels=tokenized_sent, y_labels=['Negative', 'Neutral', 'Positive'])

In [331]:
df5['sentiment'] = df5['s_spell_lem'].apply(lambda x: demo_liu_hu_lexicon2(x))

In [332]:
## Positive values mean positive sentiment, zero for neutral, negative for..negative
df5.head(20)

,s_spell_lem,category,ago,almost,also,although,always,american,another,area,...,went,whole,wing,without,working,worth,would,year,yes,sentiment
0,rocky ha been in business since the 1980s and...,others,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,their food is good,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,not haute quinine pretension,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,everyday well prepared and taste bud pleasing...,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,give mostly gone during supper hour and and a...,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,he ordered the large version of fish sandwich,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,rocky is famous for their fish,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,three large dreaded and fried fish file but a...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
8,needle to say he had no complaint and brought...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
9,i ordered the meatal drill,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [333]:
df5['business_id'] = df['Business ID']
df5['star'] = df['Review stars']

# Processing Data for Visualization

In [334]:
##Sentiments should be normalized, but whatever for this example
df5.groupby(['business_id','category']).sum().loc[:,['sentiment']]

sentiment
business_id            category           
5UmKMjUEUNdYWqANhGckJw ambience          1
                       food              3
                       others            2
                       service           3
                       value             1
KayYbHCt-RkbGcPdGOThNg ambience          5
                       food              8
                       others            3
                       service           7
                       value             2
UsFtqoBl7naz8AVUBZMjQQ ambience          4
                       fodd              0
                       food             27
                       service           1
                       value             1
WETE_LykpcnrC1sFcQ5EGg food              8
                       other             1
                       service           2
                       value             2
b9WZJp5L1RZr4F1nxclOoQ ambiance          5
                       ambience          4
                       food             46
                       other             1
                       others            3
                       service          15
                       value             5
                       wait              4
fNGIbpazjTRdXgwRY_NIXA ambience          3
                       food             15
                       others            1
                       service           1
                       value             7
mVHrayjG3uZ_RLHkLj-AMg ambience          6
                       food             26
                       others           15
                       service          20
                       value             6
                       wait              1
zaXDakTd3RXyOa7sMrUE1g ambiance          4
                       food              0
                       other             0
                       service           1
                       value             6
                       wait             -1

In [335]:
##Interesting to see how 5 stars generally have positive sentiments on all aspects
df5.groupby(['star','category']).sum().loc[:,['sentiment']]

sentiment
star category           
1.0  food              8
     value             3
3.0  ambience          3
     fodd              0
     food             13
     others            1
     service           4
     value             7
     wait              0
4.0  ambience          7
     food             29
     others           12
     service          10
     value             0
     wait              4
5.0  ambience         13
     food             61
     others           11
     service          24
     value             9
     wait              2